In [14]:
import pandas as pd
import numpy as np
from sklearn.utils import resample

In [16]:
# jiji = pd.read_csv("jiji_clean.csv")
autorec = pd.read_csv("clean_data/autorec_clean.csv")
bef = pd.read_csv("clean_data/beforward_clean.csv")

In [4]:
print("Loaded:")
# print("Jiji:", jiji.shape)
print("Autorec:", autorec.shape)
print("BeForward:", bef.shape)

Loaded:
Autorec: (50, 9)
BeForward: (2500, 16)


In [5]:
autorec["source"] = "Autorec"
bef["source"] = "BeForward"

In [6]:
final_columns = [
    "source",
    "make",
    "model",
    "year",
    "mileage",
    "engine_cc",
    "transmission",
    "fuel",
    "location",
    "price_ugx",
    "url"
]

In [ ]:
# jiji_merged = pd.DataFrame({
#     "source": jiji["source"],
#     "make": jiji["make"],
#     "model": jiji["model"],
#     "year": jiji["year"],
#     "mileage": jiji["mileage"],
#     "engine_cc": None,
#     "transmission": jiji["transmission"],
#     "fuel": jiji["fuel"],
#     "location": jiji.get("location", None),
#     "price_ugx": jiji["price"],
#     "url": jiji["url"]
# })

In [7]:
autorec_merged = pd.DataFrame({
    "source": autorec["source"],
    "make": autorec["make"],
    "model": autorec["model"],
    "year": autorec["year"],
    "mileage": autorec["mileage"],
    "engine_cc": autorec.get("engine_cc", None),
    "transmission": autorec.get("transmission", None),
    "fuel": autorec.get("fuel", None),
    "location": None,
    "price_ugx": autorec.get("price_ugx", None),
    "url": autorec["url"]
})

In [8]:
bef_merged = pd.DataFrame({
    "source": bef["source"],
    "make": bef["make"],
    "model": bef["model"],
    "year": bef["year"],
    "mileage": bef["mileage"],
    "engine_cc": bef.get("engine_cc", None),
    "transmission": bef.get("transmission", None),
    "fuel": bef.get("fuel", None),
    "location": bef.get("location", None),
    "price_ugx": bef.get("price_ugx", None),
    "url": bef["url"]
})

In [9]:
# merged = pd.concat([jiji_merged, autorec_merged, bef_merged], ignore_index=True)
merged = pd.concat([autorec_merged, bef_merged], ignore_index=True)
print("\nMerged shape before cleaning:", merged.shape)


Merged shape before cleaning: (2550, 11)


In [10]:
merged.dropna(how="all", inplace=True)
print("Merged shape after dropping empty rows:", merged.shape)

Merged shape after dropping empty rows: (2550, 11)


In [11]:
merged = merged.sort_values(["make", "model", "year"], na_position="last")

In [13]:
merged.to_csv("clean_data/cars_merged.csv", index=False)